# Imports

In [4]:
from connections.mongodb_connector import Mongo_Connector
from connections.neo4j_connector import Neo4j_Connector
import os
from datetime import datetime
from gensim import corpora, models, similarities
from models.graph_generator import Graph_Generator
from models.tuple_extractor import Tuple_Extractor
import pickle
from gensim.models.ldamulticore import LdaMulticore
import gensim
from gensim import corpora, models, similarities
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim import corpora, models, similarities
from connections.mysql_connector import MySQL_Connector
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df
from models.topic_modeling import Topic_Modeling
from gensim.models.wrappers import LdaMallet
import pandas as pd
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm  # for notebooks
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [30]:

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]


['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)


In [25]:
def gerar_tf_idf(corpus):

    vectorizer = TfidfVectorizer()
    tf_idf = vectorizer.fit_transform(corpus)
    tokens_list = vectorizer.get_feature_names()
    # print(vectorizer.get_feature_names())
    # print(tf_idf.shape)

    return tf_idf, tokens_list, vectorizer   

def get_sentencas_ids(tag_metodo):
    query = '''SELECT DISTINCT sentenca_id FROM grupo_resumo WHERE tag_metodo = "{}";'''.format(tag_metodo)
    return read(conn, query)

def get_tuplas_grupo(sentenca_id, tag_metodo):

    query = '''
    SELECT * FROM tupla_grupo tg
        INNER JOIN 
            grupo_resumo gr USING(grupo_id)
        INNER JOIN 
            tuplas tu USING(tupla_id, sentenca_id)
        WHERE 
            sentenca_id = {}
        AND
            tag_metodo = "{}";
    
    '''.format(sentenca_id, tag_metodo)
    return read(conn, query)

def closest_to_centroid_tuple(group_tuples, tuples_id):
    group_centroid = np.mean(group_tuples, axis=0)

    candidates = []
    for i, group_tuple in enumerate(group_tuples):
        euclidean_dist = np.linalg.norm(group_tuple-group_centroid)
        candidates.append([i,euclidean_dist])
    
    
    best_tuple = (min(candidates, key=lambda x: x[1]))
    tuple_id = tuples_id[best_tuple[0]]
    distance = best_tuple[1] 
    return tuple_id, distance

def get_embeddings(embedding_table, sentenca_id, tag_metodo, label):
    query = '''
    SELECT * 
        FROM
        (
            SELECT *, IF(tr.grupo_id is not NULL, "sim","nao") as processado
                FROM 
                    {} em 
                INNER JOIN
                    tupla_grupo tg USING(tupla_id)
                INNER JOIN 
                    grupo_resumo gr USING(grupo_id, sentenca_id)
                INNER JOIN 
                    tuplas tu USING(tupla_id, sentenca_id, video_id)
                LEFT JOIN
                    tupla_representativa tr USING(grupo_id, tupla_id)
                WHERE 
                    sentenca_id = {} AND
                    tag_metodo = "{}" AND
                    label = "{}"
        ) q
        WHERE processado = "nao"
        
    '''.format(embedding_table, sentenca_id, tag_metodo, label)
    df_retorno = read(conn, query)
    
    df_tuplas = df_retorno[['sentenca_id', "tupla_id", "grupo_id", "num_grupo"]]
    
    df_array = df_retorno.drop(['embedding_id','sentenca_id',
                                "tupla_id","label", 
                                "tupla_completa",
                                "rel", "arg1", 
                                "arg2","video_id",
                                "tag_metodo", "num_grupo",
                               "processado" ], axis=1) #Gera um df somente com as dimensões (vetor de caracteristicas)    
    
    array = df_array.to_numpy() #Transforma o df_array em um numpy array 
    
    series_array =  pd.Series([elemento for elemento in array]) #Gera um serie com cada linha sendo um np array
    
    df_tuplas['array'] = series_array 
    return df_tuplas, df_retorno

### Calcular Representativas por TF-IDF

In [87]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)

tag_metodo = "distancia-cosseno-80"
label = "tupla_completa"


lista_sentId = list(get_sentencas_ids(tag_metodo)['sentenca_id'])

lista_representativa = []

for sentenca_id in tqdm(lista_sentId):
    
    
    df_tuplas = get_tuplas_grupo(sentenca_id, tag_metodo)
    corpus = list(df_tuplas["tupla_completa"])
    lista_num_grupos =list(df_tuplas["num_grupo"].unique())
    
    tf_idf, tokens_list, vectorizer = gerar_tf_idf(corpus)
    
    series_tf_idf =  pd.Series([elemento.toarray() for elemento in tf_idf])
    df_tuplas['tf_idf'] = series_tf_idf 
    
    for num_grupo in lista_num_grupos:
        df_grupos = df_tuplas.loc[df_tuplas['num_grupo'] == num_grupo] 
        
        grupo_id =  df_grupos['grupo_id'].iloc[0]
      
        group_tuples = list(df_grupos['tf_idf'])
        tuples_id = list(df_grupos["tupla_id"])
        
        
        tupla_id, _ = closest_to_centroid_tuple(group_tuples, tuples_id)
        
        dict_result = {
            "grupo_id":grupo_id,
            "tupla_id":tupla_id
        }
        lista_representativa.append(dict_result)

        
df_representativo = pd.DataFrame(lista_representativa)
# replace_df(df_representativo, "tupla_representativa",conn)


### Calcular Representativas por BERT

In [38]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)

tag_metodo = "bert-cosseno-80"
label = "tupla_completa"
embedding_table = "bert"

lista_sentId = list(get_sentencas_ids(tag_metodo)['sentenca_id'])

lista_representativa = []

lista_sentId = [969]

for sentenca_id in tqdm(lista_sentId):
    
    df_tuplas, _ = get_embeddings(embedding_table, sentenca_id, tag_metodo, label)
    lista_num_grupos =list(df_tuplas["num_grupo"].unique())
    
    for num_grupo in lista_num_grupos:
        df_grupos = df_tuplas.loc[df_tuplas['num_grupo'] == num_grupo] 

        grupo_id =  df_grupos['grupo_id'].iloc[0]

        group_tuples = list(df_grupos['array'])
        tuples_id = list(df_grupos["tupla_id"])


        tupla_id, _ = closest_to_centroid_tuple(group_tuples, tuples_id)

        dict_result = {
            "grupo_id":grupo_id,
            "tupla_id":tupla_id
        }
        lista_representativa.append(dict_result)

#     df_representativo['grupo_id'] = df_representativo['grupo_id'].astype(str)
#     df_representativo['tupla_id'] = df_representativo['tupla_id'].astype(str)
    
    df_representativo = pd.DataFrame(lista_representativa)
    


Iniciando Persistencia
Infelizmente algo deu errado na inserção (persistir_uma_linha)
'numpy.int64' object has no attribute 'translate'



In [21]:
connector = MySQL_Connector("conn_orfeu")
db_name = "flairs2"
conn = connector.return_conn(db_name)


tag_metodo = "bert-cosseno-80"
label = "tupla_completa"
embedding_table = "bert"
sentenca_id = 1

df_tuplas, df_retorno = get_embeddings(embedding_table, sentenca_id, tag_metodo, label)

In [43]:
replace_df(df_representativo, "tupla_representativa",conn)

Iniciando Persistencia
Sucesso na inserção


1

In [3]:


# list(df_retorno.columns)